<a href="https://colab.research.google.com/github/sheldor07/understainding-transformers/blob/main/multilayer-perceptron-from-scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import tensorflow as tf
import numpy as np

# Loading the dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape)
print(x_test.shape)
print('----------------')
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
----------------
(60000,)
(10000,)


In [73]:
# reshaping the data to fit our input layer

x_train = x_train.reshape(x_train.shape[0],-1)/255.0
x_test = x_test.reshape(x_test.shape[0],-1)/255.0


print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [74]:
# one hot encoding labels to fit our output layer

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)
